In [42]:
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import RobustScaler, OneHotEncoder, StandardScaler, MinMaxScaler
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer,MissingIndicator
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score

In [2]:
df = pd.read_csv("C:\\Users\\arups\\OneDrive\\Desktop\\COLLECTION\\PROGRAMS\\TEST_CSVs\\income_evaluation.csv")
df.sample(10)


,age,workclass,fnlwgt,education,education-num,marital-status,occupation,relationship,race,sex,capital-gain,capital-loss,hours-per-week,native-country,income
32510,39,Private,107302,HS-grad,9,Married-civ-spouse,Prof-specialty,Husband,White,Male,0,0,45,?,>50K
9310,43,Private,352005,Bachelors,13,Married-civ-spouse,Sales,Husband,White,Male,15024,0,45,United-States,>50K
931,19,?,52114,Some-college,10,Never-married,?,Own-child,White,Female,0,0,10,United-States,<=50K
11083,49,Local-gov,193249,Assoc-acdm,12,Married-civ-spouse,Adm-clerical,Husband,White,Male,0,0,40,United-States,<=50K
21085,27,Private,214858,HS-grad,9,Married-civ-spouse,Other-service,Own-child,White,Male,0,0,40,United-States,<=50K
10502,29,Private,135296,Bachelors,13,Never-married,Exec-managerial,Not-in-family,White,Female,0,2258,45,United-States,>50K
30998,30,Private,207172,Bachelors,13,Married-civ-spouse,Adm-clerical,Wife,White,Female,0,0,40,Mexico,<=50K
17678,46,Local-gov,175754,Masters,14,Divorced,Prof-specialty,Not-in-family,White,Female,0,1876,60,United-States,<=50K
4436,39,Private,185099,HS-grad,9,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,40,United-States,<=50K
31337,44,Private,165599,HS-grad,9,Married-civ-spouse,Machine-op-inspct,Husband,White,Male,0,0,48,United-States,<=50K


In [ ]:
df.isnull().values.any()
(df == ' ?').any().any()

In [3]:
cols = list(df.columns)

for col in cols:
    
    for val in df[col].unique():
        
        if val == ' ?':
            df[col].replace(' ?' , np.nan , inplace = True)
        


In [4]:
df.isnull().sum()

age                   0
 workclass         1836
 fnlwgt               0
 education            0
 education-num        0
 marital-status       0
 occupation        1843
 relationship         0
 race                 0
 sex                  0
 capital-gain         0
 capital-loss         0
 hours-per-week       0
 native-country     583
 income               0
dtype: int64

In [5]:
df.columns = df.columns.str.strip() # This will remove extra column gap ex: '     col'
df.columns

Index(['age', 'workclass', 'fnlwgt', 'education', 'education-num',
       'marital-status', 'occupation', 'relationship', 'race', 'sex',
       'capital-gain', 'capital-loss', 'hours-per-week', 'native-country',
       'income'],
      dtype='object')

In [6]:
df.isna().sum()


age                  0
workclass         1836
fnlwgt               0
education            0
education-num        0
marital-status       0
occupation        1843
relationship         0
race                 0
sex                  0
capital-gain         0
capital-loss         0
hours-per-week       0
native-country     583
income               0
dtype: int64

## # JUST TO SPICE THINGS UP A LITTLE,I'LL INFUSE MISSING VALUES IN NUMERICAL COLUMNS AS WELL

In [7]:
np.array(df.index)

array([    0,     1,     2, ..., 32558, 32559, 32560], dtype=int64)

In [8]:
df.index

RangeIndex(start=0, stop=32561, step=1)

In [9]:
np.random.seed(15)
r = np.random.choice(df.index, size = 40 , replace = False)
print(r)
df.loc[r , 'age'] = np.nan

[10125 11478  4224  6592 21910 25737 32017 13595 16559 12752 13786 18301
 21099 12408 28467 12247 27397 26561 29923 14741 25613 26132 17718 15920
 17821 30011  8622  9580 30129 14653 26693 29771 23620  9708   807  5212
  2463 14875  4138   767]


In [10]:
np.random.seed(25)
s = np.random.choice(df.index, size = 40 , replace = False)
print(s)
df.loc[r , 'hours-per-week'] = np.nan

[ 3254 30026  7221 23481 15318 27004 32462  1033 19921 23753 18252 16482
  5790 32463  5545  5010 22026  1334 27182 28010 30808 30029  6859  5682
 23453 23589 21217 26579  7209 32356  4188 20018  1648 22219 25007 15516
 10767 20086 20713  5973]


In [11]:
df.isnull().sum()

age                 40
workclass         1836
fnlwgt               0
education            0
education-num        0
marital-status       0
occupation        1843
relationship         0
race                 0
sex                  0
capital-gain         0
capital-loss         0
hours-per-week      40
native-country     583
income               0
dtype: int64

In [12]:
X_train, X_test, y_train, y_test = train_test_split(df.drop('income', axis=1), df.income,
                                                   test_size=0.2, random_state=0)

In [13]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 32561 entries, 0 to 32560
Data columns (total 15 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   age             32521 non-null  float64
 1   workclass       30725 non-null  object 
 2   fnlwgt          32561 non-null  int64  
 3   education       32561 non-null  object 
 4   education-num   32561 non-null  int64  
 5   marital-status  32561 non-null  object 
 6   occupation      30718 non-null  object 
 7   relationship    32561 non-null  object 
 8   race            32561 non-null  object 
 9   sex             32561 non-null  object 
 10  capital-gain    32561 non-null  int64  
 11  capital-loss    32561 non-null  int64  
 12  hours-per-week  32521 non-null  float64
 13  native-country  31978 non-null  object 
 14  income          32561 non-null  object 
dtypes: float64(2), int64(4), object(9)
memory usage: 3.7+ MB


In [14]:
# Num cols

num_cols = [cols for cols in X_train.columns if X_train[cols].dtypes != 'O']

# Categorical cols

cat_cols = [cols for cols in X_train.columns if (X_train[cols].dtype == 'O') and (cols != 'education')]
# (cols != 'education') because we apply Ordinal encoding in this

In [15]:
cat_cols

['workclass',
 'marital-status',
 'occupation',
 'relationship',
 'race',
 'sex',
 'native-country']

# The not so good and unscable approach

In [16]:
clf1 = ColumnTransformer(transformers=[
    ('si_num' , SimpleImputer(strategy = 'mean' , add_indicator = True) , num_cols),
    ('rob_num' , RobustScaler() , num_cols)
] , remainder='drop')

In [17]:
pd.DataFrame(clf1.fit_transform(X_train)).head()

# HERE COLUMN INDEX 6 AND 7 ARE MISSING INDICATOR COLUMN
# 8 TO 13 IS ROBUSTSCALER COLUMN WHICH ARE BASICALLY THE ROBUST VALUE OF THE PREVIOUS 8 COLUMNS.COLUMN TRANSFORMER CONCATENATE THEM
# THIS IS A PROBLEM OF REPEATING THE COLUMN NAMES IN MULTIPLE COLUMN TRANSFORMER STEPS

,0,1,2,3,4,5,6,7,8,9,10,11,12,13
0,36.0,174308.0,7.0,0.0,0.0,40.0,0.0,0.0,-0.052632,-0.030971,-1.000000,0.0,0.0,0.0
1,35.0,198202.0,9.0,0.0,0.0,54.0,0.0,0.0,-0.105263,0.169550,-0.333333,0.0,0.0,2.8
2,38.0,52963.0,13.0,0.0,0.0,50.0,0.0,0.0,0.052632,-1.049314,1.000000,0.0,0.0,2.0
3,50.0,138270.0,9.0,0.0,0.0,40.0,0.0,0.0,0.684211,-0.333407,-0.333333,0.0,0.0,0.0
4,68.0,116903.0,11.0,0.0,2149.0,40.0,0.0,0.0,1.631579,-0.512721,0.333333,0.0,2149.0,0.0


In [18]:
clf2 = ColumnTransformer(transformers=[
    ('si_num' , SimpleImputer(strategy = 'mean' , add_indicator = True) , num_cols),
    ('si_cat' , SimpleImputer(strategy= 'constant',fill_value= 'missing',add_indicator=True) , cat_cols)
] , remainder='drop')

In [20]:
pd.DataFrame(clf2.fit_transform(X_train)).sample(10)

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17
21664,23.0,184255.0,9.0,0.0,0.0,40.0,0.0,0.0,Private,Married-civ-spouse,Sales,Wife,White,Female,United-States,False,False,False
2564,28.0,188711.0,13.0,0.0,0.0,20.0,0.0,0.0,Private,Never-married,Transport-moving,Unmarried,White,Male,United-States,False,False,False
6078,60.0,165441.0,4.0,0.0,0.0,40.0,0.0,0.0,Private,Divorced,Machine-op-inspct,Unmarried,White,Female,United-States,False,False,False
3859,39.0,208778.0,9.0,0.0,0.0,40.0,0.0,0.0,Private,Married-civ-spouse,Machine-op-inspct,Wife,White,Female,United-States,False,False,False
21422,30.0,105908.0,9.0,0.0,0.0,40.0,0.0,0.0,Private,Married-civ-spouse,Craft-repair,Husband,White,Male,United-States,False,False,False
954,21.0,259510.0,9.0,0.0,0.0,36.0,0.0,0.0,Private,Never-married,Handlers-cleaners,Own-child,White,Male,United-States,False,False,False
18234,39.0,267893.0,12.0,7298.0,0.0,40.0,0.0,0.0,Local-gov,Married-civ-spouse,Craft-repair,Husband,Black,Male,United-States,False,False,False
9416,42.0,102343.0,15.0,0.0,0.0,72.0,0.0,0.0,State-gov,Married-civ-spouse,Prof-specialty,Husband,Asian-Pac-Islander,Male,India,False,False,False
20700,31.0,262024.0,9.0,0.0,0.0,40.0,0.0,0.0,Private,Married-civ-spouse,Machine-op-inspct,Husband,Black,Male,United-States,False,False,False
19002,32.0,158416.0,9.0,0.0,0.0,50.0,0.0,0.0,Private,Divorced,Sales,Not-in-family,White,Male,United-States,False,False,False


In [30]:
clf3 = ColumnTransformer([
    ('ro_scl' , RobustScaler() , list(range(6))),
    ('ohe_ed' , OneHotEncoder(sparse_output=False, handle_unknown='ignore') , list(range(8,18)))
    
] , remainder='drop')

In [31]:
xtf = clf2.fit_transform(X_train)
clf3.fit_transform(xtf)

array([[-0.05263158, -0.03097116, -1.        , ...,  0.        ,
         1.        ,  0.        ],
       [-0.10526316,  0.16955041, -0.33333333, ...,  0.        ,
         1.        ,  0.        ],
       [ 0.05263158, -1.04931426,  1.        , ...,  0.        ,
         1.        ,  0.        ],
       ...,
       [-0.73684211, -1.11348049,  0.        , ...,  0.        ,
         1.        ,  0.        ],
       [ 0.42105263,  0.31775544,  2.        , ...,  0.        ,
         1.        ,  0.        ],
       [-0.63157895,  0.07491235,  0.        , ...,  0.        ,
         1.        ,  0.        ]])

In [27]:
clf2.fit_transform(X_train)

array([[36.0, 174308.0, 7.0, ..., False, False, False],
       [35.0, 198202.0, 9.0, ..., False, False, False],
       [38.0, 52963.0, 13.0, ..., False, False, False],
       ...,
       [23.0, 45317.0, 10.0, ..., False, False, False],
       [45.0, 215862.0, 16.0, ..., False, False, False],
       [25.0, 186925.0, 10.0, ..., False, False, False]], dtype=object)

# The Good and scalable approach

In [32]:
pp_num = Pipeline([
    ('num_imp' , SimpleImputer(strategy='median' , add_indicator=False)),
    ('rob_num' , RobustScaler())
])

pp_cat = Pipeline([
    ('cat_imp' , SimpleImputer(strategy='constant' , add_indicator=False,fill_value='missing')),
    ('ohe_cat' , OneHotEncoder(sparse_output=False , handle_unknown='ignore'))
])

In [35]:
ct = ColumnTransformer([
    ('mi' , MissingIndicator() , X_train.columns),
    ('pp_num' , pp_num , num_cols),
    ('pp_cat' , pp_cat , cat_cols)
])

In [36]:
Xt = ct.fit_transform(X_train)

In [39]:
pd.DataFrame(Xt).head()

,0,1,2,3,4,5,6,7,8,9,...,87,88,89,90,91,92,93,94,95,96
0,0.0,0.0,0.0,0.0,0.0,-0.052632,-0.030971,-1.000000,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,-0.105263,0.169550,-0.333333,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.052632,-1.049314,1.000000,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.684211,-0.333407,-0.333333,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,1.631579,-0.512721,0.333333,0.0,2149.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0


In [40]:
ct.transform(X_test)

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [41]:
pipe_final = Pipeline([
    ('ct_step' , ct),
    ('model' , DecisionTreeClassifier())
])

In [44]:
pipe_final.fit(X_train , y_train)
y_pred = pipe_final.predict(X_test)
accuracy_score(y_pred , y_test)

0.8102257024412713

In [54]:
pipe_final.named_steps['ct_step']. ['pp_cat'].named_steps['ohe_cat'].get_feature_names_out()


array(['x0_ Federal-gov', 'x0_ Local-gov', 'x0_ Never-worked',
       'x0_ Private', 'x0_ Self-emp-inc', 'x0_ Self-emp-not-inc',
       'x0_ State-gov', 'x0_ Without-pay', 'x0_missing', 'x1_ Divorced',
       'x1_ Married-AF-spouse', 'x1_ Married-civ-spouse',
       'x1_ Married-spouse-absent', 'x1_ Never-married', 'x1_ Separated',
       'x1_ Widowed', 'x2_ Adm-clerical', 'x2_ Armed-Forces',
       'x2_ Craft-repair', 'x2_ Exec-managerial', 'x2_ Farming-fishing',
       'x2_ Handlers-cleaners', 'x2_ Machine-op-inspct',
       'x2_ Other-service', 'x2_ Priv-house-serv', 'x2_ Prof-specialty',
       'x2_ Protective-serv', 'x2_ Sales', 'x2_ Tech-support',
       'x2_ Transport-moving', 'x2_missing', 'x3_ Husband',
       'x3_ Not-in-family', 'x3_ Other-relative', 'x3_ Own-child',
       'x3_ Unmarried', 'x3_ Wife', 'x4_ Amer-Indian-Eskimo',
       'x4_ Asian-Pac-Islander', 'x4_ Black', 'x4_ Other', 'x4_ White',
       'x5_ Female', 'x5_ Male', 'x6_ Cambodia', 'x6_ Canada',
       'x6_ Ch